In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

gemini_api_key = os.getenv("Gemini_api_key")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [3]:
# tool create 

@tool
def multiply(a:int, b:int)->int:
    """Given 2 number a and b this tool return their prodeuct"""
    return a*b

In [4]:
print(multiply.invoke({'a':3,'b':4}))

12


In [5]:
# tool bind
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash',api_key=gemini_api_key)

In [20]:
query = HumanMessage('can you multiply 3 with 10')

In [49]:
messages=[query]

In [50]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={})]

In [51]:
llm_with_tools = llm.bind_tools([multiply])

In [52]:
result=llm_with_tools.invoke(messages)
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--d03d282c-edc9-4aad-ba04-0beb20f5f602-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '7a4ec437-294f-43d3-865a-192978ab92a3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 30, 'output_tokens': 5, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}})

In [53]:
messages.append(result)

In [54]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--d03d282c-edc9-4aad-ba04-0beb20f5f602-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '7a4ec437-294f-43d3-865a-192978ab92a3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 30, 'output_tokens': 5, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}})]

In [56]:
tool_result = multiply.invoke(result.tool_calls[0])

In [57]:
messages.append(tool_result)

In [58]:
messages

[HumanMessage(content='can you multiply 3 with 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--d03d282c-edc9-4aad-ba04-0beb20f5f602-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 10.0}, 'id': '7a4ec437-294f-43d3-865a-192978ab92a3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 30, 'output_tokens': 5, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='30', name='multiply', tool_call_id='7a4ec437-294f-43d3-865a-192978ab92a3')]

In [59]:
llm_with_tools.invoke(messages).content

'The product of 3 and 10 is 30.'

In [47]:
from langchain_core.messages import AIMessage

final_ai_msg = [m for m in messages if isinstance(m, AIMessage)][-1]

print(final_ai_msg.content)  # e.

The product of 3 and 10 is 30.


In [48]:
messages.clear()

## Currency conversion tool

In [166]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """the function fetches the currency conversion factor between a given base currency and target currency"""
    url = f'https://v6.exchangerate-api.com/v6/8e96a032a642206bdff83fca/pair/{base_currency}/{target_currency}'
    respnse = requests.get(url)
    data =  respnse.json()
    return data['conversion_rate']

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float,InjectedToolArg()]) -> float:
    """Convert amount given a rate."""
    return base_currency_value*conversion_rate

In [167]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

88.784

In [168]:
convert.invoke({'base_currency_value':10,'conversion_rate':88.784})

887.84

In [169]:
#  tool binding

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash',api_key=gemini_api_key)


In [170]:
llm_with_tools = llm.bind_tools([get_conversion_factor,convert])

In [171]:
messages = [HumanMessage('What is the conversion factor between USD and INR,and based on that can you convert 10 inr to usd')]

In [172]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR,and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [173]:
ai_message = llm_with_tools.invoke(messages)

In [174]:
messages.append(ai_message)

In [175]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '17a519b2-078a-4063-869c-b67efd03ecd2',
  'type': 'tool_call'}]

In [176]:
import json
for tool_call in ai_message.tool_calls:
    # execute first tool and then get valiue of conversion rate
    if tool_call['name']=='get_conversion_factor':
       tool_message1 = get_conversion_factor.invoke(tool_call)
        #    print(tool_message1)
       conversion_rate = tool_message1.content
       messages.append(tool_message1)
       # second tool will be run from that conversion rate
    if tool_call['name']=='convert':
        # fetch current argument
        tool_call['args']['conversion_rate']= conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)


In [177]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 88.784. I cannot convert 10 INR to USD without knowing the inverse conversion factor or using the convert function with USD as the base currency. Would you like me to find the conversion factor between INR and USD?'

In [179]:
messages = [HumanMessage(
    "Use the get_conversion_factor tool to fetch USD → INR rate, "
    "then use convert tool to calculate 10 USD in INR."
)]
ai_message = llm_with_tools.invoke(messages)
print(ai_message.tool_calls)


[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': '0416ba5a-bfa0-4564-8d55-045df8f42de8', 'type': 'tool_call'}]
